# Current Blockchain ( 2nodes )

In [ ]:
!rm -f /tmp/*.json
!rm -f /tmp/*.pb

In [90]:
!peer lifecycle chaincode querycommitted --channelID mychannel --name scray-invoice-example --output json

{
	"sequence": 1,
	"version": "1.0",
	"endorsement_plugin": "escc",
	"validation_plugin": "vscc",
	"validation_parameter": "EiAvQ2hhbm5lbC9BcHBsaWNhdGlvbi9FbmRvcnNlbWVudA==",
	"collections": {},
	"approvals": {
		"Org1MSP": true,
		"Org2MSP": true,
		"sally0MSP": false,
		"sally1MSP": false,
		"sally2MSP": false,
		"sally3MSP": false,
		"sally4MSP": false,
		"sally5MSP": false
	}
}


# init peer access (certificates,...)

In [54]:
import os
import json
import subprocess
import pandas as pd
 
pwd0 = "/home/jovyan/work/fabric-samples/"
pwd = "/home/jovyan/work/fabric-samples/test-network"

os.environ['PATH'] = "/home/jovyan/work/fabric-samples/bin:" + str(os.environ.get('PATH')) 
os.environ['CORE_PEER_TLS_ENABLED'] = "true"
os.environ['CORE_PEER_LOCALMSPID'] = "Org1MSP"
os.environ['CORE_PEER_TLS_ROOTCERT_FILE'] = pwd + "/organizations/peerOrganizations/org1.example.com/peers/peer0.org1.example.com/tls/ca.crt"
os.environ['CORE_PEER_MSPCONFIGPATH'] = pwd + "/organizations/peerOrganizations/org1.example.com/users/Admin@org1.example.com/msp"
os.environ['CORE_PEER_ADDRESS'] = "peer0.org1.example.com:7051"
os.environ['FABRIC_CFG_PATH'] = pwd0 + "/config/"

addr0      = 'peer0.org1.example.com:7050'
orderer    = 'orderer.example.com'
ordererpem = pwd + '/organizations/ordererOrganizations/example.com/orderers/orderer.example.com/msp/tlscacerts/tlsca.example.com-cert.pem'

addr1  = 'peer0.org1.example.com:7051'
crt1   = pwd + '/organizations/peerOrganizations/org1.example.com/peers/peer0.org1.example.com/tls/ca.crt'
msp1   = pwd + '/organizations/peerOrganizations/org1.example.com/users/Admin@org1.example.com/msp'
mspid1 = "Org1MSP"

addr2  = 'peer0.org2.example.com:9051'
crt2   = pwd + '/organizations/peerOrganizations/org2.example.com/peers/peer0.org2.example.com/tls/ca.crt' 
msp2   = pwd + '/organizations/peerOrganizations/org2.example.com/users/Admin@org2.example.com/msp'
mspid2 = "Org2MSP"

def createEnvironmentForPeer(localmspid=mspid1, tlsrootcert=crt1, mspconfigpath=msp1, peeraddress=addr1):
    os.environ['CORE_PEER_LOCALMSPID']        = localmspid
    os.environ['CORE_PEER_TLS_ROOTCERT_FILE'] = tlsrootcert
    os.environ['CORE_PEER_MSPCONFIGPATH']     = mspconfigpath
    os.environ['CORE_PEER_ADDRESS']           = peeraddress
    
def createEnvironment(orgname): 
    os.environ['ORDERER_IP']="10.15.128.97"
    os.environ['ORDERER_HOSTNAME']="orderer.example.com" 
    os.environ['CHANNEL_NAME']="mychannel"
    os.environ['ORG_ID']=orgname
    os.environ['CORE_PEER_LOCALMSPID']="Org1MSP"
    os.environ['CORE_PEER_TLS_ROOTCERT_FILE']=crt1
    os.environ['CORE_PEER_MSPCONFIGPATH']=msp1
    os.environ['ORDERER_CA']=ordererpem
    os.environ['CORE_PEER_ADDRESS'] = "peer0.org1.example.com:7051"
    
def inform_existing_nodes():
    !/home/jovyan/work/scray/projects/invoice-hyperledger-fabric/scripts/inform_existing_nodes_local.sh $ORDERER_IP $CHANNEL_NAME $ORG_ID "/home/jovyan/work/fabric-samples/test-network" /home/jovyan/work/tmp
    
def example_network_install_and_approve_cc():     
    !/home/jovyan/work/scray/projects/invoice-hyperledger-fabric/scripts/example_network_install_and_approve_cc.sh 10.14.128.38 10.15.128.97 /home/jovyan/work/fabric-samples/test-network    
    
def example_network_commit_cc():    
    os.environ['IP_CC_SERVICE']='10.14.128.38'
    #Host where the example network is running
    os.environ['IP_OF_EXAMPLE_NETWORK']='10.15.128.97'
    !/home/jovyan/work/scray/projects/invoice-hyperledger-fabric/scripts/example_network_commit_cc.sh $IP_CC_SERVICE $IP_OF_EXAMPLE_NETWORK /home/jovyan/work/fabric-samples/test-network   
    
def getCurrentConfig():
    !peer channel fetch config config_block.pb -o orderer.example.com:7050 -c $CHANNEL_NAME --tls --cafile $ORDERER_CA
    !configtxlator proto_decode --input config_block.pb --type common.Block > config_block.json
    !jq .data.data[0].payload.data.config config_block.json > config.json
    
    
#!configtxlator proto_decode --type='common.Envelope' --input=/home/jovyan/work/tmp/org3_update_in_envelope.pb --output=a.json   
def convertLocalEnvelopeToJson(local_path='./', input='' ,output=''):  
    try:    
        output = subprocess.check_output(['configtxlator', 'proto_decode','--type=common.Envelope',
                                          '--input='  + local_path + '/' + input, 
                                          '--output=' + local_path + '/' + output])
        return output.decode('ascii')
    except Exception as e:
        return str(e) 

In [55]:
#getCurrentConfig()
ordererpem
!more /home/jovyan/work/fabric-samples/test-network/organizations/ordererOrganizations/example.com/orderers/orderer.example.com/msp/tlscacerts/tlsca.example.com-cert.pem

-----BEGIN CERTIFICATE-----
MIICRDCCAeqgAwIBAgIRAKKpx849GSnUqMkVpaDEbzcwCgYIKoZIzj0EAwIwbDEL
MAkGA1UEBhMCVVMxEzARBgNVBAgTCkNhbGlmb3JuaWExFjAUBgNVBAcTDVNhbiBG
cmFuY2lzY28xFDASBgNVBAoTC2V4YW1wbGUuY29tMRowGAYDVQQDExF0bHNjYS5l
eGFtcGxlLmNvbTAeFw0yMTAxMDQwODE3MDBaFw0zMTAxMDIwODE3MDBaMGwxCzAJ
BgNVBAYTAlVTMRMwEQYDVQQIEwpDYWxpZm9ybmlhMRYwFAYDVQQHEw1TYW4gRnJh
bmNpc2NvMRQwEgYDVQQKEwtleGFtcGxlLmNvbTEaMBgGA1UEAxMRdGxzY2EuZXhh
bXBsZS5jb20wWTATBgcqhkjOPQIBBggqhkjOPQMBBwNCAAT+BYJ6g04PPAI0Xg2t
4s2/ZtQrg6gEZy9uOZKMzkshbTXiU19asT8QrSrLAE5NTMzjaJlv//9wgOl7URtd
aJkFo20wazAOBgNVHQ8BAf8EBAMCAaYwHQYDVR0lBBYwFAYIKwYBBQUHAwIGCCsG
AQUFBwMBMA8GA1UdEwEB/wQFMAMBAf8wKQYDVR0OBCIEIHLdKLxxCQ9Qpkr2Auqx
Ine7n583OIk8r5+B9qa024RtMAoGCCqGSM49BAMCA0gAMEUCIQDPJhHp6OUeZhar
24gn3J1q+7mSlXiD/NnF2QUhDKWCHgIgLv8prtHKqWcvxtk/lCBO+yMUtt6Ubptq
Up9LPBu22Ok=
-----END CERTIFICATE-----


In [8]:
import ipywidgets as widgets
from IPython.display import clear_output
from IPython.display import Javascript, display
import uuid

style = {'description_width': '250px'}
layout = {'width': '500px'}

def addVisText(key='',value='',disabled=False):
    text = widgets.Text(description = key,value = value, style=style, layout=layout,disabled=disabled)
    display(text)  
    return text
    
def addVisCheckbox(key='',value=False):
    text = widgets.Checkbox(description = key,value = value, style=style, layout=layout)
    display(text)   
    return text

In [2]:
def getCrt(org='',domain='kubernetes.research.dev.seeburger.de',host='10.15.136.41:30080',user='scray', pwd='scray',channel='mychannel'):
    try:    
        output = subprocess.check_output(['curl', '--user',user + ':' + pwd, 
                                               'http://' + host + '/ca/' + channel + '/' + org + '-' + domain + '-ca.crt'])
        #return str(callProcess)[3:-3]
        return output.decode('ascii')
    except Exception as e:
        return str(e)    

def getCrtToLocal(org='',topath='/tmp',domain='kubernetes.research.dev.seeburger.de',host='10.15.136.41:30080',user='scray', pwd='scray',channel='mychannel'):
    try:    
        output = subprocess.check_output(['curl', '--user',user + ':' + pwd, 
                                               'http://' + host + '/ca/' + channel + '/' + org + '-' + domain + '-ca.crt', '-o', '/tmp/new.crt'])
        #return str(callProcess)[3:-3]
        return output.decode('ascii')
    except Exception as e:
        return str(e)   

def uploadFromLocal(host='10.15.136.41:30080',user='scray', pwd='scray',
                    local_path='./', local_name='' ,remote_path='upload'):
    try:    
        output = subprocess.check_output(['curl', '--user',user + ':' + pwd, 
                                          '-T', local_path + '/' + local_name, 
                                          'http://' + host + '/' + remote_path + '/' + local_name])
        return output.decode('ascii')
    except Exception as e:
        return str(e)   
    
def downloadToLocal(host='10.15.136.41:30080',user='scray', pwd='scray',
                    local_path='./', local_name='' ,remote_path='upload'):  
    try:    
        output = subprocess.check_output(['curl', '--user',user + ':' + pwd, 
                                               'http://' + host + '/' + remote_path + '/' + local_name, '-o', local_path + '/' + local_name])
        #return str(callProcess)[3:-3]
        return output.decode('ascii')
    except Exception as e:
        return str(e)     
    
def commit():
    try:    
        #crtnew = '\" + getCrt(org='klaus') + '\"
        getCrt(org='klaus')
        output = subprocess.check_output(['peer', 'lifecycle','chaincode','commit','-o',addr0,'--ordererTLSHostnameOverride',orderer,
                                          '--tls','--cafile',ordererpem,'--channelID','mychannel','--name','basic',
                                          '--peerAddresses',addr1, '--tlsRootCertFiles', crt1, 
                                          '--peerAddresses', addr2, '--tlsRootCertFiles', crt2, 
                                          '--peerAddresses', 'peer0.scray.org', '--tlsRootCertFiles', '/tmp/new.crt',
                                          #'--peerAddresses', 'kubernetes.research.dev.seeburger.de:32190', '--tlsRootCertFiles', '/tmp/new.crt',
                                          '--version', '1.0', '--sequence', '1','--tlsRootCertFiles'])
        return output.decode('ascii')
    except Exception as e:
        return str(e)
    
#commit()        

Create application for new node, sign it by two nodes

In [88]:
import json

def uploadConsortiumToSharepoint():
    with open('/home/jovyan/work/tmp/config.json') as f:
        d = json.load(f)
    dkeys = d.get('channel_group').get('groups').get('Application').get('groups').keys()
    with open('/home/jovyan/work/tmp/org_msp.json', 'w') as fp:
        json.dump(list(dkeys), fp)
    uploadFromLocal(local_path='/home/jovyan/work/tmp/', local_name='org_msp.json' ,remote_path='upload')

def signBy2peers():
    createEnvironmentForPeer(localmspid=mspid1, tlsrootcert=crt1, mspconfigpath=msp1, peeraddress=addr1)
    !peer channel signconfigtx -f /home/jovyan/work/tmp/org3_update_in_envelope.pb
    createEnvironmentForPeer(localmspid=mspid2, tlsrootcert=crt2, mspconfigpath=msp2, peeraddress=addr2)
    !peer channel signconfigtx -f /home/jovyan/work/tmp/org3_update_in_envelope.pb

def update():
    !peer channel update -f /home/jovyan/work/tmp/org3_update_in_envelope.pb -c $CHANNEL_NAME -o orderer.example.com:7050 --tls --cafile $ORDERER_CA

def handle_submit(text):
    global peername
    peername = text.value
    
    !rm /home/jovyan/work/tmp/*
    createEnvironment(peername)
    #!echo $ORDERER_IP $CHANNEL_NAME $ORG_ID
    inform_existing_nodes()
    uploadFromLocal(local_path='/home/jovyan/work/tmp/', local_name='org3_update_in_envelope.pb' ,remote_path='upload')
    uploadConsortiumToSharepoint()
    
peertext      = addVisText(key='name')
peertext.on_submit(handle_submit)      

Text(value='', description='name', layout=Layout(width='500px'), style=DescriptionStyle(description_width='250…

In [85]:
#createEnvironmentForPeer(localmspid=mspid1, tlsrootcert=crt1, mspconfigpath=msp1, peeraddress=addr1)
#!peer channel signconfigtx -f /home/jovyan/work/tmp/org3_update_in_envelope.pb

#signBy2peers()
uploadFromLocal(local_path='/home/jovyan/work/tmp/', local_name='org3_update_in_envelope.pb' ,remote_path='upload')
uploadConsortiumToSharepoint()
#update()

In [89]:
downloadToLocal(local_path='/home/jovyan/work/tmp/', local_name='org3_update_in_envelope.pb' ,remote_path='upload')
update()

2021-01-05 07:41:38.262 UTC [channelCmd] InitCmdFactory -> INFO 001 Endorser and orderer connections initialized
2021-01-05 07:41:38.310 UTC [channelCmd] update -> INFO 002 Successfully submitted channel update


In [64]:

uploadFromLocal(local_path='/home/jovyan/work/tmp/', local_name='org3_update_in_envelope.pb' ,remote_path='upload')


''

Testing

In [86]:
downloadToLocal(local_path='./', local_name='org3_update_in_envelope.pb' ,remote_path='upload')
convertLocalEnvelopeToJson(local_path='./', input='org3_update_in_envelope.pb' ,output='a.json')

''

In [ ]:
# load config, upload orgs list to  sharepoint upload
import json
with open('/home/jovyan/work/tmp/config.json') as f:
    d = json.load(f)
    
#print(d)
#d.keys()
dkeys = d.get('channel_group').get('groups').get('Application').get('groups').keys()

#https://stackoverflow.com/questions/7100125/storing-python-dictionaries
with open('/home/jovyan/work/tmp/org_msp.json', 'w') as fp:
    json.dump(list(dkeys), fp)
    
with open('/home/jovyan/work/tmp/org_msp.json', 'r') as fp:
    data = json.load(fp)
    
uploadFromLocal(local_path='/home/jovyan/work/tmp/', local_name='org_msp.json' ,remote_path='upload')

In [ ]:
convertLocalEnvelopeToJson(local_path='/home/jovyan/work/tmp', input='org3_update_in_envelope.pb' ,output='a.json')

with open('/home/jovyan/work/tmp/a.json') as f:
    d = json.load(f)

In [ ]:
import fakturxWidget
import ipywidgets as widgets
from IPython.display import clear_output
outs = widgets.Output()
style = {'description_width': '250px'}
layout = {'width': '500px'}

print(d.keys())
fakturxWidget.addChilds(d,'/payload',hideUnused=True)  
#d

In [ ]:
os.environ['SHARED_FS_USER'] = "scray"
os.environ['SHARED_FS_PW']   = "scray"
os.environ['SHARED_FS_HOST'] = "10.15.136.41:30080"

#!curl --user $SHARED_FS_USER:$SHARED_FS_PW -X MKCOL http://$SHARED_FS_HOST/upload/
 

uploadFromLocal(local_path='./', local_name='org3_update_in_envelope.pb' ,remote_path='upload')

In [ ]:
downloadToLocal(local_path='./', local_name='org3_update_in_envelope.pb' ,remote_path='upload')

In [ ]:
!ls ./org3_update_in_envelope.pb
#!mv org3_update_in_envelope.pb org3_update_in_envelope1.pb

In [ ]:
#curl --user $SHARED_FS_USER:$SHARED_FS_PW -X MKCOL http://$SHARED_FS_HOST/newmemberrequests/
#curl --user $SHARED_FS_USER:$SHARED_FS_PW -X MKCOL http://$SHARED_FS_HOST/newmemberrequests/$CHANNEL_NAME
#curl --user $SHARED_FS_USER:$SHARED_FS_PW -X DELETE http://$SHARED_FS_HOST/newmemberrequests/$CHANNEL_NAME/${ORG_NAME}.json
#curl --user $SHARED_FS_USER:$SHARED_FS_PW -T organizations/peerOrganizations/$DOMAINE/${ORG_NAME}.json http://$SHARED_FS_HOST/newmemberrequests/$CHANNEL_NAME/${ORG_NAME}.json
    
# Upload CA
#curl --user $SHARED_FS_USER:$SHARED_FS_PW -X MKCOL http://$SHARED_FS_HOST/ca
#curl --user $SHARED_FS_USER:$SHARED_FS_PW -X MKCOL http://$SHARED_FS_HOST/ca/$CHANNEL_NAME
#curl --user $SHARED_FS_USER:$SHARED_FS_PW -T organizations/peerOrganizations/$DOMAINE/users/User1@$DOMAINE/tls/ca.crt http://$SHARED_FS_HOST/ca/$CHANNEL_NAME/$DOMAINE-ca.crt


In [ ]:
!peer channel update -f org3_update_in_envelope.pb -c $CHANNEL_NAME -o orderer.example.com:7050 --tls --cafile $ORDERER_CA


In [ ]:
!rm -f /tmp/new.crt
res = getCrtToLocal(org='tanja2')
#res = getCrt(org='lara2')
print(res)
!ls /tmp/*.crt
commit()

In [ ]:
!/home/jovyan/work/scray/projects/invoice-hyperledger-fabric/scripts/example_network_install_and_approve_cc.sh 10.14.128.38 10.15.128.97 /home/jovyan/work/fabric-samples/test-network

In [ ]:
peer lifecycle chaincode approveformyorg 
-o orderer.example.com:7050 
--ordererTLSHostnameOverride orderer.example.com 
--tls  
--cafile /tmp/tlsca.example.com-cert.pem 
--channelID $CHANNEL_ID 
--name basic 
--version 1.0 --package-id $PKGID --sequence 1

In [ ]:
#print(getCrt(org='klaus'))

getCrtToLocal(org='tanja2')

In [ ]:
getCrt(org='klaus')
!ls /tmp

In [ ]:
os.environ['IP_CC_SERVICE']='10.14.128.38'
#Host where the example network is running
os.environ['IP_OF_EXAMPLE_NETWORK']='10.15.128.97'

!/home/jovyan/work/scray/projects/invoice-hyperledger-fabric/scripts/example_network_commit_cc.sh $IP_CC_SERVICE $IP_OF_EXAMPLE_NETWORK /home/jovyan/work/fabric-samples/test-network

In [ ]:
!cat /home/jovyan/work/scray/projects/invoice-hyperledger-fabric/scripts/example_network_commit_cc.sh

# old

In [ ]:

!peer channel fetch config /tmp/config_block.pb -o orderer.example.com:7050 -c $CHANNEL_NAME --tls --cafile $ORDERER_CA > /dev/null
!configtxlator proto_decode --input /tmp/config_block.pb --type common.Block | jq .data.data[0].payload.data.config > /tmp/config.json   

In [ ]:
# Upload CA cert
os.environ['SHARED_FS_HOST']="10.15.136.41:30080"
os.environ['SHARED_FS_USER']="scray"
os.environ['SHARED_FS_PW']="scray"
#!apk add curl

os.environ['NEW_ORG_NAME']="OrgSally"

!curl --user $SHARED_FS_USER:$SHARED_FS_PW -X MKCOL http://$SHARED_FS_HOST/ca
!curl --user $SHARED_FS_USER:$SHARED_FS_PW -T /home/jovyan/work/fabric-samples/test-network/organizations/ordererOrganizations/example.com/orderers/orderer.example.com/msp/tlscacerts/tlsca.example.com-cert.pem http://$SHARED_FS_HOST/ca/tlsca.example.com-cert.pem

# Get configuration of new peer
#curl --user 'scray:scray' http://${SHARED_FS_HOST}/newmemberrequests/$CHANNEL_NAME/${NEW_ORG_NAME}.json > new_member_org.json
!curl --user $SHARED_FS_USER:$SHARED_FS_PW  http://${SHARED_FS_HOST}/newmemberrequests/$CHANNEL_NAME/${NEW_ORG_NAME}.json > /tmp/new_member_org.json 
        
# Add org3 data to existing config
!jq -s ".[0] "*" {\"channel_group\":{\"groups\":{\"Application\":{\"groups\": {\"${NEW_ORG_NAME}MSP\":.[1]}}}}}" /tmp/config.json /tmp/new_member_org.json > /tmp/modified_config.json

!configtxlator proto_encode --input /tmp/config.json --type common.Config --output /tmp/config.pb
!configtxlator proto_encode --input /tmp/modified_config.json --type common.Config --output /tmp/modified_config.pb
!configtxlator compute_update --channel_id $CHANNEL_NAME --original /tmp/config.pb --updated /tmp/modified_config.pb --output /tmp/org3_update.pb
!configtxlator proto_decode --input /tmp/org3_update.pb --type common.ConfigUpdate | jq . > /tmp/org3_update.json
!echo '{"payload":{"header":{"channel_header":{"channel_id":"'$CHANNEL_NAME'", "type":2}},"data":{"config_update":'$(cat /tmp/org3_update.json)'}}}' | jq . > /tmp/org3_update_in_envelope.json
!configtxlator proto_encode --input /tmp/org3_update_in_envelope.json --type common.Envelope --output /tmp/org3_update_in_envelope.pb        

sign and update

In [ ]:
!peer channel signconfigtx -f /tmp/org3_update_in_envelope.pb

os.environ['CORE_PEER_LOCALMSPID']="Org2MSP"
os.environ['CORE_PEER_TLS_ROOTCERT_FILE']=crt2
os.environ['CORE_PEER_MSPCONFIGPATH']=msp2
os.environ['ORDERER_CA']=ordererpem
os.environ['CORE_PEER_ADDRESS'] = addr2

!peer channel update -f /tmp/org3_update_in_envelope.pb -c $CHANNEL_NAME -o orderer.example.com:7050 --tls --cafile $ORDERER_CA

# network_install_and_approve_cc (3 nodes)

In [94]:
#!pwd
#!ls /home/jovyan/work/scray/projects/invoice-hyperledger-fabric/scripts/example_network_install_and_approve_cc.sh
!cd /tmp
!/home/jovyan/work/scray/projects/invoice-hyperledger-fabric/scripts/example_network_install_and_approve_cc.sh 10.14.128.38 10.15.128.97 /home/jovyan/work/fabric-samples/test-network

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   450  100   450    0     0   1840      0 --:--:-- --:--:-- --:--:--     04      0 --:--:-- --:--:-- --:--:--  1844
2021-01-05 08:07:00.017 UTC [cli.lifecycle.chaincode] submitInstallProposal -> INFO 001 Installed remotely: response:<status:200 payload:"\nJbasic_1.0:5a294a12a1a89cd4eed3d4234fbc79f42eab2ac20cd176bc8ebbc07c597cd0ee\022\tbasic_1.0" > 
2021-01-05 08:07:00.017 UTC [cli.lifecycle.chaincode] submitInstallProposal -> INFO 002 Chaincode code package identifier: basic_1.0:5a294a12a1a89cd4eed3d4234fbc79f42eab2ac20cd176bc8ebbc07c597cd0ee
2021-01-05 08:07:00.140 UTC [cli.lifecycle.chaincode] submitInstallProposal -> INFO 001 Installed remotely: response:<status:200 payload:"\nJbasic_1.0:5a294a12a1a89cd4eed3d4234fbc79f42eab2ac20cd176bc8ebbc07c597cd0ee\022\tbasic_1.0" > 
2021-01-05 08:07:00.140 UTC [cli.lifecycle.chaincode] 

# commit

In [97]:
os.environ['IP_CC_SERVICE']='10.14.128.38'
#Host where the example network is running
os.environ['IP_OF_EXAMPLE_NETWORK']='10.15.128.97'

!/home/jovyan/work/scray/projects/invoice-hyperledger-fabric/scripts/example_network_commit_cc.sh $IP_CC_SERVICE $IP_OF_EXAMPLE_NETWORK /home/jovyan/work/fabric-samples/test-network

Error: proposal failed with status: 500 - failed to invoke backing implementation of 'CommitChaincodeDefinition': requested sequence is 1, but new definition must be sequence 2
2021-01-05 21:26:01.156 UTC [chaincodeCmd] chaincodeInvokeOrQuery -> INFO 001 Chaincode invoke successful. result: status:200 


# check network (approved chaincode)

In [98]:
!peer lifecycle chaincode querycommitted --channelID mychannel --name basic --output json

#!peer lifecycle chaincode checkcommitreadiness --channelID mychannel --name basic --output json --version  1.0 --sequence 2

#!peer lifecycle chaincode --channelID mychannel
#!discover --configFile conf.yaml peers --channel mychannel  --server peer0.org1.example.com:7051

#!discover --configFile config.json peers --channel mychannel  --server peer0.org1.example.com:7051

{
	"sequence": 1,
	"version": "1.0",
	"endorsement_plugin": "escc",
	"validation_plugin": "vscc",
	"validation_parameter": "EiAvQ2hhbm5lbC9BcHBsaWNhdGlvbi9FbmRvcnNlbWVudA==",
	"collections": {},
	"approvals": {
		"Org1MSP": true,
		"Org2MSP": true,
		"sally0MSP": true,
		"sally1MSP": true,
		"sally2MSP": true,
		"sally3MSP": true,
		"sally4MSP": true,
		"sally5MSP": true
	}
}


In [99]:
!peer lifecycle chaincode querycommitted --channelID mychannel --name scray-invoice-example --output json
!peer lifecycle chaincode queryinstalled

{
	"sequence": 1,
	"version": "1.0",
	"endorsement_plugin": "escc",
	"validation_plugin": "vscc",
	"validation_parameter": "EiAvQ2hhbm5lbC9BcHBsaWNhdGlvbi9FbmRvcnNlbWVudA==",
	"collections": {},
	"approvals": {
		"Org1MSP": true,
		"Org2MSP": true,
		"sally0MSP": false,
		"sally1MSP": false,
		"sally2MSP": false,
		"sally3MSP": false,
		"sally4MSP": false,
		"sally5MSP": false
	}
}
Installed chaincodes on peer:
Package ID: scray-invoice-example_1.0:00183e641cfcb303d7a6236a8461500223224ad3d06c75f68a6a32338dc6415f, Label: scray-invoice-example_1.0
Package ID: basic_1.0:5a294a12a1a89cd4eed3d4234fbc79f42eab2ac20cd176bc8ebbc07c597cd0ee, Label: basic_1.0


In [ ]:
#!peer chaincode query -C mychannel -n scray-invoice-example -c '{"Args":["get","name"]}'
#./network.sh createChannel -c channelone
!peer channel list
!peer lifecycle chaincode querycommitted --channelID channelone --name scray-invoice-example --output json

In [ ]:
!env | fgrep MSP

# Test

In [100]:
def readAsset(asset='asset1'):
    callopt= '{\"function\":\"ReadAsset\",\"Args\":[' + '\"' + asset + '\"' + ']}'
    try:    
        callProcess  = subprocess.check_output(['peer', 'chaincode','query', '-C', 'mychannel', '-n', 'basic', '-c', callopt])
        return json.loads(str(callProcess)[2:-3])
    except Exception as e:
        return str(e)
    
readAsset('asset1')    

'Command \'[\'peer\', \'chaincode\', \'query\', \'-C\', \'mychannel\', \'-n\', \'basic\', \'-c\', \'{"function":"ReadAsset","Args":["asset1"]}\']\' returned non-zero exit status 1.'

In [103]:
readAsset('asset2')  

'Command \'[\'peer\', \'chaincode\', \'query\', \'-C\', \'mychannel\', \'-n\', \'basic\', \'-c\', \'{"function":"ReadAsset","Args":["asset2"]}\']\' returned non-zero exit status 1.'

In [102]:
readAsset('asset4')  

'Command \'[\'peer\', \'chaincode\', \'query\', \'-C\', \'mychannel\', \'-n\', \'basic\', \'-c\', \'{"function":"ReadAsset","Args":["asset4"]}\']\' returned non-zero exit status 1.'

In [101]:
!peer chaincode query -C mychannel -n basic -c '{"function":"GetAllAssets","Args":[]}'

# REST

In [ ]:
!peer channel fetch config config_block.pb -o orderer.example.com:7050 -c $CHANNEL_NAME --tls --cafile $ORDERER_CA
!configtxlator proto_decode --input config_block.pb --type common.Block > config_block.json
    

In [ ]:
!peer channel fetch config /tmp/config_block.pb -o orderer.example.com:7050 -c $CHANNEL_NAME --tls --cafile $ORDERER_CA > /dev/null  

In [ ]:
#peer channel fetch config config_block.pb -o orderer.example.com:7050 -c $CHANNEL_NAME --tls --cafile $ORDERER_CA
import sys    
def channelFetch(channelName,ordererpem):
    #rint(channelName,ordererpem)
    try:    
        output = subprocess.check_output(['peer', 'channel','fetch', 'config', 'config_block.pb', '-o', 'orderer.example.com:7050', '-c',channelName, '--tls', 
                                               '--cafile', ordererpem], stderr=subprocess.STDOUT, shell=True, timeout=3,universal_newlines=True)
        #output = subprocess.check_output(['peer', 'channel','fetch', 'config', 'config_block.pb', '-o', 'orderer.example.com:7050', '-c',channelName, '--tls', 'cafile', ordererpem], stderr=subprocess.STDOUT, shell=True, timeout=3,universal_newlines=True)
        #output = subprocess.check_output(['peer', 'channel','fetch','newest', 'config', '/tmp/config_block.pb', '-o', 'orderer.example.com:7050', '-c',channelName, '--tls', '--cafile', ordererpem], stderr=subprocess.STDOUT)
        
    #xcept Exception as e:
    #   print('exception' + str(e))
    #   return None
    except subprocess.CalledProcessError as exc:
        print("Status : FAIL", exc.returncode, exc.output)
    else:
        print("Output: \n{}\n".format(output))
    
#channelFetch('mychannel',ordererpem)    

#!cd /home/jovyan/work/
#rm -f config_block.pb
channelFetch('mychannel',ordererpem)
